### Initial look at clustering for Asteroid tracklets:

In [1]:
# Imports
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import inconsistent
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import numpy as np


np.set_printoptions(precision=5, suppress=True)
%matplotlib inline

In [7]:
with open('../ITF/test_r2.5_rd0.out','r') as file:
    lines = file.read().split('\n')
    r2p5 = [i.split() for i in lines]



1031